<a href="https://colab.research.google.com/github/BibhuDas123/Deep_learning/blob/master/Muticlassification_tensorflow_model_with_minoirty_class_handling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import,division,print_function,unicode_literals
import tensorflow as tf
import os,datetime
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true',compression='zip',sep=',',quotechar='"')

In [3]:
df.columns
pd.set_option('display.max_colwidth',1)


In [4]:
X_train,X_test=train_test_split(df,test_size=0.2,random_state=0)

In [5]:
#Handling unbalanved classes in the target but applying new methos:class weight
#what it odes it adds majoirty weight to minority classes and vice versa
from sklearn.utils import class_weight
class_weights=list(class_weight.compute_class_weight('balanced',
                                                    np.unique(df['Product']),
                                                    df['Product']))

In [6]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                       9799 
Checking or savings account    7003 
Student loan                   2950 
Vehicle loan or lease          2736 
Name: Product, dtype: int64

In [7]:
class_weights.sort()
class_weights

[0.43980801028844385,
 0.7258015614340938,
 0.9771915501581794,
 1.3673425674710837,
 3.2459322033898306,
 3.4998172514619883]

In [8]:
weights={}
for index,weight in enumerate(class_weights):
  weights[index]=weight


In [9]:
dataset_train=tf.data.Dataset.from_tensor_slices((X_train['Consumer complaint narrative'].values,X_train['Product'].values))
dataset_test=tf.data.Dataset.from_tensor_slices((X_test['Consumer complaint narrative'].values,X_test ['Product'].values))

In [10]:
for text,target in dataset_train.take(5):
  print('Complaint:{},Target:{}'.format(text,target))

Complaint:b'am contacting Bank Of America in reference due to a balance owed from a overdraft checkings account. I am requesting for a typed document which stated the amounts owed and the also the account number associated with the overdraft amount. I am requesting for the account to be deleted from XXXX, The XXXX  XXXX  XXXX and XXXX XXXX and any other reporting system. For an agreement or exchange to pay this balance on my reports from years ago. The derogatory information you have provided in reference to this account is visible on my consumer report/file with the agencys listed above. As this is damaging my chances with any future banking relationship with other banks. Please acknowledge this written statement and send a balance and payment option document to the address you have on file or that submitted with this dispute.',Target:b'Checking or savings account'
Complaint:b'On XXXX XXXX XXXX XXXX sent {$1000.00} to my XXXX  XXXX  which had my bank account linked. On XXXX XXXX XXXX 

In [11]:
for text,target in dataset_test.take(5):
  print('Complaint:{},Target;{}'.format(text,target))

Complaint:b'My loan officer, XXXX XXXX, with Mortgage Express, and realtor, XXXX XXXX, told me to apply for an owner occupied loan, when they both knew that I wanted to rent the property out, in order to take care of my XXXX year old mother. The first loan I applied for with Mortgage Express was for a rental property loan. My offer was not accepted. When I found another home I put an offer on, they told me since I would eventually live in the home ( when my mother passed away ), to apply for a owner occupied loan. When the loan closed, and I informed my home owners insurance company ( XXXX ), they told me I needed a rental policy not home owners. When I contacted XXXX XXXX she stated she knew nothing about home insurance. I plan to occupy this home AND take care of my XXXX year old mother in her home, but I do not want Mortgage Express to do this to anyone else. XXXX XXXX has left threatening messages on my cell phone when I have asked to speak to her manager. She has been trying to in

In [12]:
table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(
        keys=tf.constant(['Debt collection', 'Credit card or prepaid card', 'Mortgage', 'Checking or savings account', 'Student loan', 'Vehicle loan or lease']),
        values=tf.constant([0, 1, 2, 3, 4, 5]),
    ),
    default_value=tf.constant(-1),
    name="target_encoding"
)

@tf.function
def target(x):
  return table.lookup(x)

In [13]:
def show_batch(dataset,size=5):
  for batch,label in dataset.take(size):
    print(batch.numpy())
    print(target(label).numpy())

In [14]:
show_batch(dataset_test,6)

b'My loan officer, XXXX XXXX, with Mortgage Express, and realtor, XXXX XXXX, told me to apply for an owner occupied loan, when they both knew that I wanted to rent the property out, in order to take care of my XXXX year old mother. The first loan I applied for with Mortgage Express was for a rental property loan. My offer was not accepted. When I found another home I put an offer on, they told me since I would eventually live in the home ( when my mother passed away ), to apply for a owner occupied loan. When the loan closed, and I informed my home owners insurance company ( XXXX ), they told me I needed a rental policy not home owners. When I contacted XXXX XXXX she stated she knew nothing about home insurance. I plan to occupy this home AND take care of my XXXX year old mother in her home, but I do not want Mortgage Express to do this to anyone else. XXXX XXXX has left threatening messages on my cell phone when I have asked to speak to her manager. She has been trying to intimidate m

In [15]:
#creation of one hot encoding
def fetch(text,labels):
  return text,tf.one_hot(target(labels),6)

train_data_f=dataset_train.map(fetch)
test_data_f=dataset_test.map(fetch)

In [16]:
next(iter(train_data_f))
train_data,train_labels=next(iter(train_data_f.batch(5)))
train_data,train_labels

(<tf.Tensor: shape=(5,), dtype=string, numpy=
 array([b'am contacting Bank Of America in reference due to a balance owed from a overdraft checkings account. I am requesting for a typed document which stated the amounts owed and the also the account number associated with the overdraft amount. I am requesting for the account to be deleted from XXXX, The XXXX  XXXX  XXXX and XXXX XXXX and any other reporting system. For an agreement or exchange to pay this balance on my reports from years ago. The derogatory information you have provided in reference to this account is visible on my consumer report/file with the agencys listed above. As this is damaging my chances with any future banking relationship with other banks. Please acknowledge this written statement and send a balance and payment option document to the address you have on file or that submitted with this dispute.',
        b'On XXXX XXXX XXXX XXXX sent {$1000.00} to my XXXX  XXXX  which had my bank account linked. On XXXX XXXX 

In [17]:

embedding = "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"
hub_layer = hub.KerasLayer(embedding, output_shape=[128], input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_data[:1])

<tf.Tensor: shape=(1, 128), dtype=float32, numpy=
array([[ 1.3365566e+00,  1.4613692e-01, -4.2354032e-02, -2.5150621e-01,
        -1.1561170e-01,  1.5430607e-01, -8.3720177e-02,  2.3264247e-01,
         1.2643227e-01,  4.1582039e-01,  1.6804273e-01, -2.2835983e-01,
        -2.7798966e-01,  1.3967808e-01, -1.4639112e-01,  2.5525475e-02,
        -2.2495736e-01,  1.2668982e-02, -4.3704787e-01,  1.2598745e+00,
        -5.4309763e-02,  1.2522198e-01,  1.7858028e-02,  2.3717348e-01,
        -3.3231676e-02, -3.6916888e-01,  8.3625101e-02, -2.0063482e-01,
        -1.0708701e-01, -4.9423046e-02, -7.8526877e-02, -1.8613932e-01,
        -2.3709236e-01, -2.4819662e-01,  2.1108375e-01,  7.4445799e-02,
        -2.8739634e-01, -2.4010691e-01, -4.7701992e-02, -1.3101815e-01,
         2.6113200e-06, -3.5220775e-01, -1.4700818e-01, -5.6435522e-02,
         1.9818431e-01,  5.6152910e-02, -5.2523676e-02, -9.3921766e-02,
        -5.9601028e-02, -3.3253166e-01,  6.4379163e-02, -1.8356338e-01,
         2.465

In [21]:
model=tf.keras.Sequential()
model.add(hub_layer)
for units in [128,128,64,32]:
  model.add(tf.keras.layers.Dense(units,activation='relu'))
  model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(6,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 128)               124642688 
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_5 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_6 (Dropout)          (None, 64)               

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [23]:
train_data_f=train_data_f.shuffle(70000).batch(512)
test_data_f=test_data_f.batch(512)

In [25]:
#when we give a class weight is uses a weighted class entory function and basically evaluates how good the classifier is per label rather than across all lables
history=model.fit(train_data_f,
                  epochs=50,
                  validation_data=test_data_f,
                  verbose=1,
                  class_weight=weights)

Epoch 1/50
90/90 [==============================] - 9s 101ms/step - loss: 1.6710 - accuracy: 0.3073 - val_loss: 1.3840 - val_accuracy: 0.6492
Epoch 2/50
90/90 [==============================] - 9s 99ms/step - loss: 1.3171 - accuracy: 0.7450 - val_loss: 1.1985 - val_accuracy: 0.8462
Epoch 3/50
90/90 [==============================] - 9s 100ms/step - loss: 1.1882 - accuracy: 0.8525 - val_loss: 1.1865 - val_accuracy: 0.8564
Epoch 4/50
90/90 [==============================] - 9s 100ms/step - loss: 1.1514 - accuracy: 0.8833 - val_loss: 1.1789 - val_accuracy: 0.8638
Epoch 5/50
90/90 [==============================] - 9s 99ms/step - loss: 1.1374 - accuracy: 0.8975 - val_loss: 1.1792 - val_accuracy: 0.8632
Epoch 6/50
90/90 [==============================] - 9s 100ms/step - loss: 1.1286 - accuracy: 0.9069 - val_loss: 1.1679 - val_accuracy: 0.8755
Epoch 7/50
90/90 [==============================] - 9s 100ms/step - loss: 1.1210 - accuracy: 0.9137 - val_loss: 1.1772 - val_accuracy: 0.8655
Epoch 8/

In [27]:

results=model.evaluate(dataset_test.map(fetch).batch(len(list(dataset_test))),verbose=2)
print(results)

1/1 - 0s - loss: 1.1708 - accuracy: 0.8718
[1.1708319187164307, 0.8718127012252808]


In [28]:
#the batch value is given to but it take the max value of records avaialble in batch
test_data,test_labels=next(iter(dataset_test.map(fetch).batch(45963)))

In [29]:
y_pred=model.predict(test_data)

In [30]:
from sklearn.metrics import classification_report
#Using argmax=1 as we are are using fetch fiunction which does one hot encoding.
print(classification_report(test_labels.numpy().argmax(axis=1),y_pred.argmax(axis=1)))

              precision    recall  f1-score   support

           0       0.93      0.89      0.91      4312
           1       0.82      0.88      0.85      2642
           2       0.89      0.93      0.91      1994
           3       0.84      0.81      0.83      1362
           4       0.82      0.81      0.82       595
           5       0.73      0.73      0.73       586

    accuracy                           0.87     11491
   macro avg       0.84      0.84      0.84     11491
weighted avg       0.87      0.87      0.87     11491



In [31]:
#One can use the this two data and compare wwith the results using without using class weights and see the difference.
#The class weights basically assgns weightage to different calsses giving more weightage to minority and lesser to majority classes.The majority are penalized and the minority classes is giving more importasnve so the classes of these minority classes can be better.
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels.numpy().argmax(axis=1),y_pred.argmax(axis=1))

array([[3828,  177,  119,   41,   71,   76],
       [ 128, 2321,   27,  138,    7,   21],
       [  28,   43, 1853,   19,   18,   33],
       [  29,  185,   28, 1105,    3,   12],
       [  36,   35,   24,    2,  483,   15],
       [  55,   63,   25,    9,    6,  428]])